In [49]:
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline
import matplotlib.pyplot as plt
import time
import torch
from tqdm import tqdm
from sklearn import metrics

In [50]:
class ChessData(torch.utils.data.Dataset):
    def __init__(self, path):
        self.df = pd.read_csv(path, index_col=0)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        inputs = row[:321].values.astype(np.float32)
        outputs = row[321:].values.astype(np.float32)

        return {'board': inputs, 'themes': outputs}


In [51]:
from torch.utils.data import DataLoader

batch_size = 256

chess_data_set = ChessData('cleaned_data/cleaned_train_puzzles.csv')

train_dataloader = DataLoader(chess_data_set, batch_size=batch_size, num_workers=4, persistent_workers=True, shuffle=True)

In [52]:
chess_validation_set = ChessData('cleaned_data/cleaned_validation_puzzles.csv')

validation_dataloader = DataLoader(chess_validation_set, batch_size=batch_size, num_workers=4, persistent_workers=True)

In [68]:
import torch.nn as nn

net = nn.Sequential(
    nn.Linear(321, 128),
    nn.BatchNorm1d(128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),
    nn.ReLU(),
    nn.Linear(64, 28),
)


In [69]:
optim = torch.optim.Adam(net.parameters(), lr=1e-3)

loss_fn = nn.BCEWithLogitsLoss()

In [ ]:
def get_loss():
    net.eval()
    total_loss = 0
    num_batches = 0

    for batch in tqdm(validation_dataloader):
        X = batch['board'].cuda()

        with torch.no_grad():
            pred = net(X)

        loss = loss_fn(pred, batch['themes'].cuda())
        total_loss += loss

        num_batches += 1

    total_loss /= num_batches

    return total_loss

In [70]:
net = net.cuda()
net.train()
loss_list = []

for epoch in range(100):
    epoch_train_loss = 0
    num_batches = 0

    for batch in tqdm(train_dataloader):
        X = batch['board'].cuda()

        pred = net(X)
        loss = loss_fn(pred, batch['themes'].cuda())
        epoch_train_loss += loss

        optim.zero_grad()
        loss.backward()

        optim.step()
  
        num_batches += 1

    epoch_train_loss /= num_batches
    epoch_val_loss = get_loss()

    loss_list.append((epoch_train_loss.item(), epoch_val_loss.item()))

    if epoch % 10 == 0:
        torch.save(net.state_dict(), f"/content/drive/My Drive/Faculdade/TCC/model/model_epoch_{epoch}.pt") # f"/content/drive/My Drive/Faculdade/TCC/model/model_epoch_{epoch}.pt"


100%|██████████| 2428/2428 [00:47<00:00, 50.59it/s]


In [72]:
#metrics.accuracy_score(y_true, y_pred > 0.5)


0.27066667524761395